In [1]:
import pandas as pd
import numpy as np
from urllib import parse
from urllib import request
import bs4 as bs

In [2]:
html_path = '../data/scraping/sunnyvale.html'
post_data_path = '../data/scraping/POST_sunnyvale.txt'

In [3]:
# specify location of table
url = 'https://sunnyvaleca.legistar.com/Calendar.aspx'
table_class = '.rgMasterTable'
table_id = 'ctl00_ContentPlaceHolder1_gridCalendar_ctl00'

In [6]:
# prepare POST data
with open(post_data_path) as f:
    post_data = f.read()
replacements = [('\n', '&'), (' ', ''), (':', '=')]
for a, b in replacements:
    post_data = post_data.replace(a,b)
post_data = post_data + '&ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl02$ctl00$ctl10='
post_data = post_data.encode('ascii')

In [7]:
# download web page
post_response = request.urlopen(url=url, data=post_data)
html = post_response.read()
with open(html_path, 'wb') as f:
    f.write(html)

In [ ]:
def page_arg(page_num):
    return 'ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl02$ctl00$ctl{:02d}'.format(page_num*2)
    

In [ ]:
javascript:__doPostBack('ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl02$ctl00$ctl22','')

In [10]:
javascript:__doPostBack('ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl02$ctl00$ctl22','')

In [13]:
page_arg(20)

'ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl02$ctl00$ctl40'

In [6]:
# find table in page

soup = bs.BeautifulSoup(html)
table = soup.select(table_id)[0]
num_cols = int(table.td.get('colspan'))

# extract column headers
header_data = [''.join(cell.stripped_strings) for cell in table.find_all('th')]
header_data = [h for h in header_data if h!='Data pager']
assert(len(header_data)==num_cols)

# extract text and URL data from table
text_data, url_data = [], []
for row in table.find_all('tr'):
    row_text, row_url = [], []
    for td in row.find_all('td'):
        row_text.append(''.join(td.stripped_strings))
        if td.find('a'):
            row_url.append(td.a.get('href'))
        else:
            row_url.append(np.nan)
        if len(row_text)==num_cols and len(row_url)==num_cols:
            text_data.append(row_text)
            url_data.append(row_url)

IndexError: list index out of range

In [40]:
# turn into dataframe

num_cols = table.td.get('colspan')
text_df = pd.DataFrame(text_data, columns=header_data)
url_df = pd.DataFrame(url_data, columns=header_data)
df = pd.merge(text_df, url_df, left_index=True, right_index=True, suffixes=(' Text', ' URL'))

In [41]:
df

,Name Text,Meeting Date Text,Text,Meeting Time Text,Meeting Location Text,Meeting Details Text,Agenda Text,Minutes Text,Audio/Video Text,Audio Text,...,Meeting Date URL,URL,Meeting Time URL,Meeting Location URL,Meeting Details URL,Agenda URL,Minutes URL,Audio/Video URL,Audio URL,Caption Notes URL
0,City Council,2/25/2021,,8:30 AM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733769&GUID=88628981-2EEF-44...,NaN,NaN,None,None,None,None,None,None
1,City Council,2/23/2021,,7:00 PM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733768&GUID=F88F11F1-AE4B-44...,NaN,NaN,None,None,None,None,None,None
2,City Council,2/2/2021,,7:00 PM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733767&GUID=AA5EFD4F-4E7A-4D...,NaN,NaN,None,None,None,None,None,None
3,City Council,1/28/2021,,8:30 AM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733766&GUID=FEEC36BD-0BF6-4E...,NaN,NaN,None,None,None,None,None,None
4,City Council,1/26/2021,,7:00 PM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733765&GUID=AF3AE1D7-DD8D-40...,NaN,NaN,None,None,None,None,None,None
5,City Council,1/12/2021,,7:00 PM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733764&GUID=C2EAC5EA-DD8E-4F...,NaN,NaN,None,None,None,None,None,None
6,City Council,1/5/2021,,7:00 PM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733758&GUID=AECC5500-0D31-4D...,NaN,NaN,None,None,None,None,None,None
7,Housing and Human Services Commission,12/23/2020,,7:00 PM,"West Conference Room, City Hall, 456 W. Olive ...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=726298&GUID=581C7BBF-F065-44...,NaN,NaN,None,None,None,None,None,None
8,Housing and Human Services Commission,12/16/2020,,7:00 PM,"West Conference Room, City Hall, 456 W. Olive ...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=726291&GUID=D2473D62-810A-45...,NaN,NaN,None,None,None,None,None,None
9,City Council,12/8/2020,,7:00 PM,"Council Chambers and West Conference Room, Cit...",Meeting details,Not available,Not available,Not available,Not available,...,NaN,View.ashx?M=IC&ID=733757&GUID=63225169-B8AB-4E...,NaN,NaN,None,None,None,None,None,None


In [16]:
javascript:__doPostBack('ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl03$ctl01$ctl12')

AttributeError: 'str' object has no attribute 'find_all'

In [33]:
table

<table border="0" class="rgMasterTable" frame="void" id="ctl00_ContentPlaceHolder1_gridCalendar_ctl00" style="table-layout:auto;empty-cells:show;" width="100%">
<colgroup>
<col/>
<col/>
<col/>
<col/>
<col/>
<col/>
<col/>
<col/>
<col/>
<col/>
<col/>
</colgroup>
<thead>
<tr class="rgPager">
<td colspan="11"><table border="0" style="width:100%;border-spacing:0;" summary="Data pager which controls on which page is the RadGrid control.">
<caption>
<span style="display: none">Data pager</span>
</caption><thead>
<tr>
<th scope="col">Data pager</th>
</tr>
</thead><tbody>
<tr>
<td class="rgPagerCell NumericPages"><div class="rgWrap rgNumPart">
<a class="rgCurrentPage" href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl02$ctl00$ctl02','')" onclick="return false;"><span>1</span></a><a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$gridCalendar$ctl00$ctl02$ctl00$ctl04','')"><span>2</span></a><a href="javascript:__doPostBack('ctl00$ContentPlaceHolder1$gridCalen

In [13]:

link_fields = [
    'Meeting Details',
    'Agenda',
    'Minutes',
    'Audio/Video',
    'Audio',
    'Caption Notes'
    ]

In [21]:
post_response.read()

b''

In [ ]:
cell_html = """
<td><font face="Tahoma" size="2">
    <span style="white-space: nowrap;">
        <img id="ctl00_ContentPlaceHolder1_gridCalendar_ctl00_ctl32_imgAgenda" src="Images/PDF.gif" alt="Agenda" border="0">
        
        <a id="ctl00_ContentPlaceHolder1_gridCalendar_ctl00_ctl32_hypAgenda" href="View.ashx?M=A&amp;ID=651877&amp;GUID=CBCE9D3D-CE13-40A5-B781-1EB3305F4EBC" target="_blank"><font face="Tahoma" color="Blue" size="2">Agenda</font></a>
    </span>
</font></td>
"""
soup.select('#ctl00_ContentPlaceHolder1_gridCalendar_ctl00_ctl32_hypAgenda')[0].get('href')